# Title
**Author**:  Greg Slater <br>
**Date**:  13 Jan 2025 <br>
**Dataset Scope**: `dataset` <br>
**Report Type**: Ad-hoc analysis <br>

## Purpose
[Jira ticket](https://mhclgdigital.atlassian.net/browse/DATA-1199)   
[Mural board](https://app.mural.co/t/mhclg2837/m/mhclg2837/1706786112750/e94eeff744863fe859b2d22a3ccb2dac1e442270?wid=0-1736957406208) mapping out problem and approach.   

Analysis to identify BFL entities which can be given an end-date on the basis that their references do not appear on the latest resource for a provision.

In [1]:
import pandas as pd
import os
import urllib
import sqlite3
from datetime import datetime
import matplotlib.pyplot as plt

td = datetime.today().strftime('%Y-%m-%d')

pd.set_option("display.max_rows", 100)

data_dir = "../../data/db_downloads/"
os.makedirs(data_dir, exist_ok=True)
out_dir = "../../data/deleted_entities/"
os.makedirs(out_dir, exist_ok=True)


In [2]:
def datasette_query(db, sql_string):
    params = urllib.parse.urlencode({
        "sql": sql_string,
        "_size": "max"
        })
    url = f"https://datasette.planning.data.gov.uk/{db}.csv?{params}"
    df = pd.read_csv(url)
    return df

def get_all_organisations():
    q = """
        select organisation, name, entity as organisation_entity, end_date
        from organisation
--        where end_date = ""
        """
    return datasette_query("digital-land", q)

FILES_URL = 'https://datasette.planning.data.gov.uk/'

def download_dataset(dataset, output_dir_path, overwrite=False):
    dataset_file_name = f'{dataset}.db'
    
    if not os.path.exists(output_dir_path):
        os.makedirs(output_dir_path)
    
    output_file_path = os.path.join(output_dir_path, dataset_file_name)

    if overwrite is False and os.path.exists(output_file_path):
        return
    
    final_url = os.path.join(FILES_URL, dataset_file_name)
    print(f'downloading data from {final_url}')
    print(f'to: {output_file_path}')
    urllib.request.urlretrieve(final_url, os.path.join(output_dir_path, dataset_file_name))
    print('download complete')

def query_sqlite(db_path, query_string):

    with sqlite3.connect(db_path) as con:
            
        cursor = con.execute(query_string)
        cols = [column[0] for column in cursor.description]
        results_df = pd.DataFrame.from_records(data=cursor.fetchall(), columns=cols)

    return results_df

In [3]:
# function to get historic endpoints and resources for a provision, along with incrementing resource key
def get_historic_resources(dataset, organisation):

    q = f"""

        WITH endpoint_count as (
            SELECT 
                distinct organisation,
                latest_status, 
                endpoint, 
                endpoint_entry_date, 
                endpoint_end_date, 
                resource, 
                resource_start_date, 
                resource_end_date,
                dense_rank() over (order by endpoint_entry_date, endpoint) as endpoint_no

            FROM reporting_historic_endpoints
            WHERE pipeline = '{dataset}'
            AND organisation = '{organisation}'
            AND latest_status = 200
            ORDER BY endpoint_entry_date, resource_start_date
        ),

        endpoint_resource_count as (
            SELECT 
                *,
                CAST(endpoint_no as string) || "." || dense_rank() over (partition by endpoint_no order by resource_start_date) as endpoint_resource_no
            FROM endpoint_count
        )

        SELECT 
            *,
            CASE WHEN resource_end_date = "" then 
                LEAD(endpoint_entry_date) over (order by endpoint_resource_no) 
                ELSE resource_end_date end
                as entity_end_date

        FROM endpoint_resource_count

"""
    r = datasette_query("performance", q)

    return r 

# get_historic_resources("brownfield-land", "local-authority:BST")

In [4]:
def get_resource_references(resources, db_path):

    # take list of resources and db path to get all reference values which have appeared on each resource

    q = """ 

        SELECT distinct fr.resource, f.value 
        FROM fact_resource fr
        INNER JOIN fact f on fr.fact = f.fact
        WHERE 1=1
            AND f.field = "reference"
            AND fr.resource in ({})
    """.format(', '.join(f"'{r}'" for r in resources))

    df = query_sqlite(db_path, q)
    
    return df

## Data Import

In [8]:
# download performance db
download_dataset("brownfield-land", data_dir, overwrite=True)
bfl_path = os.path.join(data_dir, "brownfield-land.db")

downloading data from https://datasette.planning.data.gov.uk/brownfield-land.db
to: ../../data/db_downloads/brownfield-land.db
download complete


In [9]:
org_lookup = get_all_organisations()

org_dict = dict(zip(org_lookup["organisation"], org_lookup["organisation_entity"]))

In [10]:
prov_bfl = datasette_query(
    "digital-land",
    """
    SELECT organisation
    FROM provision
    WHERE dataset = "brownfield-land"
--    AND end_date = ""
    """
)

len(prov_bfl)

353

## Analysis steps

Expected steps:
* For each organisation, get table of: organisation, resource, start-date, reference
* Take the latest (non-200, with more than 0 entities) resource for each organisation
* For any references from not latest resources, use end-date as end-date for entities with that reference


NOTE - we should only run this for organisations supplying authoritative data, as otherwise it will get confusing factoring in orgs like GLA who can supply data for multiple organisations. This method of working out latest entities relies on a model of on organisation supplying successive data about the same things through one or multiple endpoints.

NOTE - some organisations are using end-date (e.g. Bristol), we should make sure to only apply this calculated end date to entities which don't already have one from the org.

Questions:
* Is taking resource with latest start date robust enough to identify latest snapshot of data? What if an endpoint for 2022 data has a mistake corrected which generates a newer resource than an endpoint with 2024 data has? Should we use the latest resource from latest endpoint?

Suggest here we actually begin giving old endpoints for orgs an end date, it will make the logic of working out what end-date to give old entities SO much easier.

* What about orgs like Bucks who have multiple active endpoints to supply data for multiple regions? Can't reliably identify latest data between this model and single dataset model. May have to manually exclude orgs like Bucks, or maybe restrict method to only orgs with single active endpoint?

* how to deal with entities with data from old resources that don't appear in reporting or collection tables? e.g. Bristol entity [1725409](https://www.planning.data.gov.uk/entity/1725409) has reference facts from resource `01307bca0ecc5c950ef9b35c02ee7b3378cd9b7d29078229e4984a88d4f04ed1`. This appears in the `resource.csv` with an end-date, but isn't in `old-resource.csv`, or in the BFL `dataset_resource` or `fact_resource` tables. So not easy to work out an end-date for it from just the sqlite file. Will probably need to separately check for any more of these and then they can just be retired.


Risks:
* Using endpoint_entry_date as a proxy for the data lineage will be wrong in some cases, e.g. [Brighton and Hove](https://datasette.planning.data.gov.uk/performance?sql=SELECT+*%2C%0D%0A++++CAST%28endpoint_no+as+string%29+%7C%7C+%22.%22+%7C%7C+dense_rank%28%29+over+%28partition+by+endpoint_no+order+by+resource_start_date%29+as+endpoint_resource_no%0D%0A%0D%0AFROM+%28%0D%0ASELECT+distinct+organisation%2C+organisation_name%2C+dataset%2C+latest_status%2C+endpoint_url%2C+endpoint%2C+endpoint_entry_date%2C+endpoint_end_date%2C+resource%2C+resource_start_date%2C+resource_end_date%2C%0D%0A+++dense_rank%28%29+over+%28order+by+endpoint_entry_date%2C+endpoint%29+as+endpoint_no%0D%0AFROM+reporting_historic_endpoints%0D%0AWHERE+pipeline+%3D+%22brownfield-land%22%0D%0AAND+organisation+%3D+%22local-authority%3ABNH%22%0D%0AAND+latest_status+%3D+200%0D%0AORDER+BY+endpoint_entry_date%2C+resource_start_date%0D%0A++%29) got 2017 and 2018 data added on the same day in 2019. So references which appeared in one and not the other may get an unusual end-date.

## Provision analysis

In [ ]:
prov_bfl

In [87]:
bfl_prov = datasette_query(
    "performance",
    """
    with ep_res as (
        SELECT
        organisation,
        name,
        endpoint,
        endpoint_entry_date,
        endpoint_end_date,
        resource_start_date,
        resource
        FROM
        reporting_historic_endpoints
        WHERE
        latest_status = 200
--        AND endpoint_end_date = ""
        AND pipeline = "brownfield-land"
        AND resource != ""
    )
    
    SELECT
    organisation, name,
    ROUND((julianday(date('now')) - julianday(min(resource_start_date))) / 365, 1)  as oldest_resource_age_years,
    ROUND((julianday(date('now')) - julianday(max(resource_start_date))) / 365, 1)  as newest_resource_age_years,
    count(distinct endpoint) as n_unique_endpoints,
    count(distinct resource) as n_unique_resources
    FROM ep_res
    GROUP BY 1, 2"""
)


In [ ]:
bfl_prov.head()

In [ ]:
print(bfl_prov["n_unique_endpoints"].mean())
print(bfl_prov["n_unique_endpoints"].median())
print(bfl_prov["n_unique_endpoints"].max())

print(len(bfl_prov))
print(len(bfl_prov[bfl_prov["n_unique_endpoints"] > 1]))

In [ ]:
print(f"n BFL provisions with active endpoints: {len(bfl_prov)}")

fig, ax1 = plt.subplots(figsize=(8,5))
bfl_prov["n_unique_endpoints"].hist(bins=12)

plt.title("n active endpoints per BFL provision")
plt.xlabel("n active endpoints")
plt.ylabel("frequency")

In [ ]:
bfl_org_count = query_sqlite("../../data/db_downloads/brownfield-land.db", 
             """
                SELECT count(*) as count 
                FROM entity
             """)

In [ ]:
old_orgs = org_lookup[org_lookup["end_date"].notnull()]


bfl_prov[["organisation", "name"]].merge(
    old_orgs[["organisation", "organisation_entity"]],
    how = "inner",
    on = "organisation"
)

### Test

In [ ]:
# just testing steps with one organisation for POC - Bristol
bs_test = get_historic_resources("brownfield-land", "local-authority:BST")

# get historic resources and the endpoint_resource count key for Bristol
bs_test.head()


In [ ]:
# get all reference values for each historic resource
res_refs = get_resource_references(bs_test["resource"].drop_duplicates(), "../../data/db_downloads/brownfield-land.db")

res_refs.head()

In [ ]:
# join bristol resources to list of reference values per resource
org_res_refs = bs_test.merge(
    res_refs,
    how = "left",
    on = "resource"
)

# get the endpoint_resource_no for the most recent resource the reference value appeared in 
ref_latest_res = org_res_refs.groupby(["value"], as_index = False).agg(
    endpoint_resource_no = ("endpoint_resource_no", "max")
)

# get the endpoint_resource_no for the latest bristol resource
max_ep_res = org_res_refs["endpoint_resource_no"].max()

# old refs are those where the most recent resource they appeared on is not the latest resource
old_refs = ref_latest_res[ref_latest_res["endpoint_resource_no"] != max_ep_res]

# join back to res_refs table to get the end date of the resource each old ref last appeared on - this will be the entity end-date
old_refs_dated = org_res_refs.merge(
    old_refs,
    how = "inner",
    on = ["value", "endpoint_resource_no"]
)[["organisation", "value", "entity_end_date"]].drop_duplicates()

old_refs_dated.rename(
    inplace = True,
    columns = {"value" : "reference"}
)

old_refs_dated

In [ ]:
bst_entity = query_sqlite("../../data/db_downloads/brownfield-land.db", 
             """
                SELECT * 
                FROM entity
                WHERE organisation_entity = 66
             """)


# find entities that have a reference which doens't appear in any valid resources at all
old_ents = bst_entity[["entity", "entry_date", "reference"]].merge(
                old_refs_dated,
                how = "inner",
                on = "reference"
            )

print(f"No of bristol entities: {len(bst_entity)}")
print(f"No of old bristol entities: {len(old_ents)}")
old_ents.head()

In [ ]:
# mystery entities:
# find entities that have a reference which doens't appear in any valid resources at all
mystery_ents = bst_entity[~bst_entity["reference"].isin(res_refs["value"].drop_duplicates())]
print(f"No of bristol mystery entities: {len(mystery_ents)}")
mystery_ents.head()

### Scale: Apply to all BFL provisions

In [11]:
old_refs_all = []
old_ents_all = []
org_res_refs_all = []
mystery_ents_all = []

for org in prov_bfl["organisation"]:
    
    # get all valid resources
    res_hist = get_historic_resources("brownfield-land", org)

    print("---------------------------------------")
    print(org)
    print(f"Number of endpoint & resource records found: {len(res_hist)}")

    if len(res_hist) > 1:


        # get entities
        org_entities = query_sqlite("../../data/db_downloads/brownfield-land.db", 
            f"""
                SELECT * 
                FROM entity
                WHERE organisation_entity = {org_dict[org]}
                and end_date = ""
            """)


        print(f"Number of live entities: {len(org_entities)}")

        # get all reference values for resources
        res_refs = get_resource_references(
            res_hist["resource"].drop_duplicates(), 
            "../../data/db_downloads/brownfield-land.db")
        
        unique_refs = res_refs["value"].drop_duplicates()
        print(f"Number of unique references: {len(unique_refs)}")
        
        # join resources to list of reference values per resource
        org_res_refs = res_hist.merge(
            res_refs,
            how = "left",
            on = "resource"
        )

        org_res_refs_all.append(org_res_refs)

        # get the endpoint-resource count code for the most recent resource each reference has appeared in 
        ref_latest_res = org_res_refs.groupby(["value"], as_index = False).agg(
            endpoint_resource_no = ("endpoint_resource_no", "max")
        )

        # get the endpoint-resource count code for the latest resource
        max_ep_res = org_res_refs["endpoint_resource_no"].max()

        # old refs are those where the most recent resource they appeared on is not the latest resource
        old_refs = ref_latest_res[ref_latest_res["endpoint_resource_no"] != max_ep_res]

        print(f"Number of old references found = {len(old_refs)} ")

        # check for any entities that have a reference which doesn't appear in the list from all resources
        mystery_ents = org_entities[~org_entities["reference"].isin(unique_refs)]

        if len(mystery_ents) > 0:
            print(f"Number of mystery entities found: {len(mystery_ents)}")
            mystery_ents_all.append(mystery_ents)

        if len(old_refs) > 0:

            # join back to res_refs table to get the end date of the resource each old ref last appeared on - this will be the entity end-date
            old_refs_dated = org_res_refs.merge(
                old_refs,
                how = "inner",
                on = ["value", "endpoint_resource_no"]
            )[["organisation", "value", "entity_end_date"]].drop_duplicates()

            old_refs_dated.rename(
                inplace = True,
                columns = {"value" : "reference"}
            )

            old_refs_all.append(old_refs_dated)
            
            old_entities = org_entities.merge(
                old_refs_dated,
                how = "inner",
                on = "reference"
            )

            print(f"Number of entities to end-date : {len(old_entities)}")

            old_ents_all.append(old_entities)


---------------------------------------
development-corporation:Q117149370
Number of endpoint & resource records found: 0
---------------------------------------
development-corporation:Q124604981
Number of endpoint & resource records found: 0
---------------------------------------
development-corporation:Q20648596
Number of endpoint & resource records found: 2
Number of live entities: 79
Number of unique references: 41
Number of old references found = 0 
Number of mystery entities found: 38
---------------------------------------
development-corporation:Q6670544
Number of endpoint & resource records found: 8
Number of live entities: 3
Number of unique references: 0
Number of old references found = 0 
Number of mystery entities found: 3
---------------------------------------
development-corporation:Q72463795
Number of endpoint & resource records found: 0
---------------------------------------
local-authority:ADU
Number of endpoint & resource records found: 5
Number of live entities:

In [12]:
# org_entities[~org_entities["reference"].isin(unique_refs)]

In [13]:
# pd.concat(old_refs_all).to_csv(os.path.join(out_dir, f"test - old refs.csv"), index = False)
# pd.concat(old_ents_all).to_csv(os.path.join(out_dir, f"test - old ents.csv"), index = False)
# pd.concat(org_res_refs_all).to_csv(os.path.join(out_dir, f"test - resource refs.csv"), index = False)
# pd.concat(mystery_ents_all).to_csv(os.path.join(out_dir, f"test - mystery ents.csv"), index = False)


In [14]:
old_refs_all_df = pd.concat(old_refs_all)
old_refs_all_df.to_csv(os.path.join(out_dir, f"test - old refs - {td}.csv"), index = False)

old_ents_all_df = pd.concat(old_ents_all)
old_ents_all_df.to_csv(os.path.join(out_dir, f"test - old ents - {td}.csv"), index = False)

org_res_refs_all_df = pd.concat(org_res_refs_all)
org_res_refs_all_df.to_csv(os.path.join(out_dir, f"test - resource refs - {td}.csv"), index = False)

mystery_ents_all_df = pd.concat(mystery_ents_all)
mystery_ents_all_df.to_csv(os.path.join(out_dir, f"test - mystery ents - {td}.csv"), index = False)

old_refs_all_df["value"] = old_refs_all_df["reference"]
org_res_refs_all_df["resource_start_date"] = pd.to_datetime(org_res_refs_all_df["resource_start_date"])

print(len(org_res_refs_all_df))
print(len(old_refs_all_df))
print(len(old_ents_all_df))
print(len(mystery_ents_all_df))


167129
18476
10086
2123


### Investigating results

In [ ]:
org_res_refs_all_df.head()

In [ ]:
old_ents_all_df.head()

In [ ]:
org_res_refs_all_df[org_res_refs_all_df["organisation"] == "local-authority:HAS"]

In [ ]:
mystery_ents_all_df[mystery_ents_all_df["organisation_entity"] == "46"]


In [ ]:
print(len(org_res_refs_all_df))
print(len(old_ents_all_df))

old_ents_all_df["value"] = old_ents_all_df["reference"]

old_ref_resource_starts = org_res_refs_all_df.merge(
    old_ents_all_df[["organisation", "value", "reference"]],
    how = "inner",
    on = ["organisation", "value"]
).groupby(["organisation", "reference"], as_index=False).agg(
    resource_start_date = ("resource_start_date", "max")
)

print(len(old_ref_resource_starts))

In [ ]:

fig, ax1 = plt.subplots(figsize=(8,5))
old_ref_resource_starts["resource_start_date"].hist()

plt.title("latest resource start date for entities with old references")
plt.xlabel("latest resource start date")
plt.ylabel("frequency")

In [ ]:
# count ents per org from 2020
old_ref_resource_starts[
    (old_ref_resource_starts['resource_start_date']>pd.Timestamp(2020,1,1)) & 
    (old_ref_resource_starts['resource_start_date']<pd.Timestamp(2020,12,31))].value_counts("organisation")

In [ ]:
print(len(org_res_refs_all_df))
print(len(old_refs_all_df))

old_refs_all_df["value"] = old_refs_all_df["reference"]

old_ref_resource_starts = org_res_refs_all_df.merge(
    old_refs_all_df,
    how = "inner",
    on = ["organisation", "value"]
).groupby(["organisation", "reference"], as_index=False).agg(
    resource_start_date = ("resource_start_date", "max")
)

print(len(old_ref_resource_starts))

In [ ]:
old_ref_resource_starts["resource_start_date"].hist()

In [ ]:
res_temp = org_res_refs_all_df[org_res_refs_all_df["organisation"] == "development-corporation:Q6670544"]
res_temp
# get_resource_references(
#     res_temp["resource"].drop_duplicates(), 
#     "../../data/db_downloads/brownfield-land.db")

In [ ]:
temp_org_ent = org_dict["development-corporation:Q6670544"]

temp_entity = query_sqlite("../../data/db_downloads/brownfield-land.db", 
             f"""
                SELECT * 
                FROM entity
                WHERE organisation_entity = {temp_org_ent}
             """)

temp_entity

In [ ]:
print(len(old_refs_all_df))
print(len(old_refs_all_df["reference"].drop_duplicates()))
old_refs_all_df.head()

In [ ]:
print(len(old_ents_all_df))
print(len(old_ents_all_df["entity"].drop_duplicates()))
old_ents_all_df.head()

In [ ]:
org_res_ref_hist = pd.concat(org_res_refs_all)

org_res_ref_hist[org_res_ref_hist["organisation"] == "local-authority:ASH"]

In [ ]:
len(org_res_refs["value"].drop_duplicates())

In [94]:
# find entities that have a reference which doens't appear in any valid resources at all
# bst_entity[~bst_entity["reference"].isin(org_res_refs["value"].drop_duplicates())]